Polygon Extraction, Filtering & Save to GeoJson

In [3]:
# Script 1: fetch_and_save_postal_codes.py
import osmnx as ox
import geopandas as gpd
import json
from shapely.geometry import MultiPolygon, Polygon


def fetch_and_save_postal_codes(place_name="Germany", tags={"boundary": "postal_code"}, save_file=True):
    # Set the default CRS to EPSG:25832
    ox.settings.default_crs = "epsg:25832"

    # Fetch place polygons
    place_polygons = ox.features_from_place(place_name, tags)

    # Filter for both Polygons and MultiPolygons
    polygon_mask = place_polygons['geometry'].apply(lambda geom: isinstance(geom, (Polygon, MultiPolygon)))

    # Create GeoDataFrame and drop rows with missing postal_code or geometry
    gdf = gpd.GeoDataFrame(place_polygons[polygon_mask].dropna(subset=['postal_code', 'geometry']))

    # Keep only the necessary columns
    columns_to_keep = ['geometry', 'postal_code']
    gdf = gdf[columns_to_keep]

    # Convert and store each row's data
    mongo_data_list = []
    for idx, row in gdf.iterrows():
        postal_code = row['postal_code']
        geometry = row['geometry']

        # Convert Shapely geometry to GeoJSON format
        geojson_geometry = json.loads(gpd.GeoSeries(geometry).to_json())['features'][0]['geometry']

        # Create a dictionary with postal code and GeoJSON geometry
        postal_code_data = {
            "postal_code": postal_code,
            "geometry": geojson_geometry
        }

        # Add the dictionary to the list
        mongo_data_list.append(postal_code_data)

    # Save GeoJSON data to a file for further use
    if save_file:
        with open(f'{place_name}_postal_codes.geojson', 'w') as file:
            json.dump({"type": "FeatureCollection", "features": mongo_data_list}, file, indent=2)

    return mongo_data_list

# Example usage
result_data = fetch_and_save_postal_codes(place_name="Germany", tags={"boundary": "postal_code"}, save_file=True)
